# Implementing Neural Networks

The World Health Organization (WHO)’s Global Health Observatory (GHO) data repository tracks life expectancy for countries worldwide by following health status and many other related factors.

Although there have been a lot of studies undertaken in the past on factors affecting life expectancy considering demographic variables, income composition, and mortality rates, it was found that the effects of immunization and human development index were not taken into account.

This dataset covers a variety of indicators for all countries from 2000 to 2015 including:

* Immunization factors<br>
* Mortality factors<br>
* Economic factors<br>
* Social factors<br>
* Other health-related factors<br>

Ideally, this data will eventually inform countries concerning which factors to change in order to improve the life expectancy of their populations. If we can predict life expectancy well given all the factors, this is a good sign that there are some important patterns in the data. Life expectancy is expressed in years, and hence it is a number. This means that in order to build a predictive model one needs to use regression.

In [41]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [12]:
dataset = pd.read_csv('life.csv', index_col=0)
dataset.head().T

,0,1,2,3,4
Country,Afghanistan,Afghanistan,Afghanistan,Afghanistan,Afghanistan
Year,2015,2014,2013,2012,2011
Status,Developing,Developing,Developing,Developing,Developing
Adult Mortality,263.0,271.0,268.0,272.0,275.0
infant deaths,62,64,66,69,71
Alcohol,0.01,0.01,0.01,0.01,0.01
percentage expenditure,71.279624,73.523582,73.219243,78.184215,7.097109
Hepatitis B,65.0,62.0,64.0,67.0,68.0
Measles,1154,492,430,2787,3013
BMI,19.1,18.6,18.1,17.6,17.2


In [13]:
dataset.columns

Index(['Country', 'Year', 'Status', 'Adult Mortality', 'infant deaths',
       'Alcohol', 'percentage expenditure', 'Hepatitis B', 'Measles ', ' BMI ',
       'under-five deaths ', 'Polio', 'Total expenditure', 'Diphtheria ',
       ' HIV/AIDS', 'GDP', 'Population', ' thinness  1-19 years',
       ' thinness 5-9 years', 'Income composition of resources', 'Schooling',
       'Life expectancy'],
      dtype='object')

In [11]:
dataset.describe().head().T

,count,mean,std,min,25%
Year,2938.0,2.007519e+03,4.613841e+00,2000.00000,2004.000000
Adult Mortality,2938.0,1.647257e+02,1.240862e+02,1.00000,74.000000
infant deaths,2938.0,3.030395e+01,1.179265e+02,0.00000,0.000000
Alcohol,2938.0,4.546875e+00,3.921946e+00,0.01000,1.092500
percentage expenditure,2938.0,7.382513e+02,1.987915e+03,0.00000,4.685343
Hepatitis B,2938.0,8.302212e+01,2.299698e+01,1.00000,82.000000
Measles,2938.0,2.419592e+03,1.146727e+04,0.00000,0.000000
BMI,2938.0,3.838118e+01,1.993537e+01,1.00000,19.400000
under-five deaths,2938.0,4.203574e+01,1.604455e+02,0.00000,0.000000
Polio,2938.0,8.261777e+01,2.336717e+01,3.00000,78.000000


In [14]:
dataset = dataset.drop(['Country'], axis=1)

In [20]:
dataset.shape

(2938, 21)

In [15]:
dataset.columns

Index(['Year', 'Status', 'Adult Mortality', 'infant deaths', 'Alcohol',
       'percentage expenditure', 'Hepatitis B', 'Measles ', ' BMI ',
       'under-five deaths ', 'Polio', 'Total expenditure', 'Diphtheria ',
       ' HIV/AIDS', 'GDP', 'Population', ' thinness  1-19 years',
       ' thinness 5-9 years', 'Income composition of resources', 'Schooling',
       'Life expectancy'],
      dtype='object')

In [22]:
dataset.dtypes

Year                                 int64
Status                              object
Adult Mortality                    float64
infant deaths                        int64
Alcohol                            float64
percentage expenditure             float64
Hepatitis B                        float64
Measles                              int64
 BMI                               float64
under-five deaths                    int64
Polio                              float64
Total expenditure                  float64
Diphtheria                         float64
 HIV/AIDS                          float64
GDP                                float64
Population                         float64
 thinness  1-19 years              float64
 thinness 5-9 years                float64
Income composition of resources    float64
Schooling                          float64
Life expectancy                    float64
dtype: object

##### Train-Test-Split

In [18]:
features = dataset.iloc[:, 0:-1]
labels = dataset.iloc[:, -1]

In [19]:
features.shape

(2938, 20)

In [23]:
features = pd.get_dummies(features)

In [26]:
features.head()

,Year,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,...,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Status_Developed,Status_Developing
0,2015,263.0,62,0.01,71.279624,65.0,1154,19.1,83,6.0,...,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1,0,1
1,2014,271.0,64,0.01,73.523582,62.0,492,18.6,86,58.0,...,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0,0,1
2,2013,268.0,66,0.01,73.219243,64.0,430,18.1,89,62.0,...,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9,0,1
3,2012,272.0,69,0.01,78.184215,67.0,2787,17.6,93,67.0,...,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8,0,1
4,2011,275.0,71,0.01,7.097109,68.0,3013,17.2,97,68.0,...,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5,0,1


In [28]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.25, random_state=1)

In [30]:
numerical_features = features.select_dtypes(include=['float64', 'int64'])
numerical_columns = numerical_features.columns
 
ct = ColumnTransformer([("only numeric", StandardScaler(), numerical_columns)], remainder='passthrough')

In [31]:
features_train_scaled = ct.fit_transform(features_train)
features_test_scaled = ct.fit_transform(features_test)

In [35]:
my_model = Sequential()
input = InputLayer(input_shape=(features.shape[1], ))

In [36]:
my_model.add(input)

In [38]:
my_model.add(Dense(64, activation = 'relu'))

In [39]:
my_model.add(Dense(1))

In [40]:
my_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                1408      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,473
Trainable params: 1,473
Non-trainable params: 0
_________________________________________________________________


In [42]:
opt = Adam(learning_rate = 0.01)

In [43]:
my_model.compile(loss='mse', metrics=['mae'], optimizer=opt)

In [44]:
my_model.fit(features_train_scaled, labels_train, epochs=40, batch_size=1, verbose=1)

Epoch 1/40
2203/2203 [==============================] - 16s 6ms/step - loss: 145.5892 - mae: 6.4606
Epoch 2/40
2203/2203 [==============================] - 12s 5ms/step - loss: 32.2807 - mae: 4.0684
Epoch 3/40
2203/2203 [==============================] - 11s 5ms/step - loss: 23.3817 - mae: 3.6630
Epoch 4/40
2203/2203 [==============================] - 11s 5ms/step - loss: 23.1688 - mae: 3.4900
Epoch 5/40
2203/2203 [==============================] - 12s 5ms/step - loss: 20.4780 - mae: 3.3108
Epoch 6/40
2203/2203 [==============================] - 11s 5ms/step - loss: 17.3015 - mae: 3.0304
Epoch 7/40
2203/2203 [==============================] - 11s 5ms/step - loss: 17.0225 - mae: 3.0096
Epoch 8/40
2203/2203 [==============================] - 11s 5ms/step - loss: 16.5698 - mae: 2.9702
Epoch 9/40
2203/2203 [==============================] - 11s 5ms/step - loss: 13.3126 - mae: 2.7434
Epoch 10/40
2203/2203 [==============================] - 11s 5ms/step - loss: 14.3120 - mae: 2.6934
Epoch 11

In [46]:
res_mse, res_mae = my_model.evaluate(features_test_scaled, labels_test, verbose=0)

In [47]:
print(res_mse, res_mae)

6.806833267211914 1.8508808612823486
